In [1]:
import os
import escher
import cobra

In [2]:
sbmls_path = "/mnt/d/data/GEMs/Com_8"
files = [os.path.join(sbmls_path, file) for file in os.listdir(sbmls_path)]
files

['/mnt/d/data/GEMs/Com_8/Clostridium_ramosum_VPI_0427_DSM_1402.xml']

In [3]:
sb_model = cobra.io.read_sbml_model(files[0])

In [4]:
sb_model

Name,M_Clostridium_ramosum_VPI_0427_DSM_1402
Memory address,7f26d51688b0
Number of metabolites,962
Number of reactions,1049
Number of genes,893
Number of groups,68
Objective expression,1.0*biomass525 - 1.0*biomass525_reverse_5c178
Compartments,"Cytoplasm, Extracellular"


In [5]:
sb_model_json = cobra.io.to_json(sb_model)
builder = escher.Builder(map_name="C. ramosum", map_json=sb_model_json)

/home/josua/miniconda3_linux/envs/FBA/lib/python3.10/site-packages/escher/plots.py:537: UserWarning: map_json overrides map_name
  warn('map_json overrides map_name')


In [6]:
builder

Builder()